In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import torch
import torch.nn as nn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#USING ALREADY EXTRACTED DATA FROM JINS SENSORS
train_msAcc = np.load("/content/drive/MyDrive/Datasets/train_JinsMSAccelerometer_OpenDoor_RubHands.npy")
train_msGyro = np.load("/content/drive/MyDrive/Datasets/train_JinsMSGyroscope_OpenDoor_RubHands.npy")
train_labels = np.load("/content/drive/MyDrive/Datasets/train_Jinslabels_OpenDoor_RubHands.npy")

test_msAcc = np.load("/content/drive/MyDrive/Datasets/test_MSJinsAccelerometer_OpenDoor_RubHands.npy")
test_msGyro = np.load("/content/drive/MyDrive/Datasets/test_MSJinsGyroscope_OpenDoor_RubHands.npy")
test_labels = np.load("/content/drive/MyDrive/Datasets/test_Jinslabels_OpenDoor_RubHands.npy")

In [5]:
print(train_msAcc.shape)
print(train_msGyro.shape)
print(train_labels.shape)

print(test_msAcc.shape)
print(test_msGyro.shape)
print(test_labels.shape)

(87, 80, 3)
(87, 80, 3)
(87,)
(90, 80, 3)
(90, 80, 3)
(90,)


In [6]:
train_num_examples = 87
test_num_examples = 90

# Number of sensors and features
num_features = 8  # mean, max, min, std,precentile,zero crossing for each sensor
num_sensors = 2  # Accelerometer and Gyroscope

# Initialize arrays to store features
train_features = np.zeros((train_num_examples, num_sensors * num_features, 3))
test_features = np.zeros((test_num_examples, num_sensors * num_features, 3))

In [7]:
# Training Accelerometer features
train_features[:, 0, :] = np.mean(train_msAcc, axis=1)
train_features[:, 1, :] = np.max(train_msAcc, axis=1)
train_features[:, 2, :] = np.min(train_msAcc, axis=1)
train_features[:, 3, :] = np.std(train_msAcc, axis=1)
train_features[:, 4, :] = np.percentile(train_msAcc, 20, axis=1)
train_features[:, 5, :] = np.percentile(train_msAcc, 50, axis=1)
train_features[:, 6, :] = np.percentile(train_msAcc, 80, axis=1)
train_features[:, 7, :] = np.sum(np.diff(np.sign(train_msAcc), axis=1) != 0, axis=1)


# Training Gyroscope features
train_features[:, 8, :] = np.mean(train_msGyro, axis=1)
train_features[:, 9, :] = np.max(train_msGyro, axis=1)
train_features[:, 10, :] = np.min(train_msGyro, axis=1)
train_features[:, 11, :] = np.std(train_msGyro, axis=1)
train_features[:, 12, :] = np.percentile(train_msGyro, 20, axis=1)
train_features[:, 13, :] = np.percentile(train_msGyro, 50, axis=1)
train_features[:, 14, :] = np.percentile(train_msGyro, 80, axis=1)
train_features[:, 15, :] = np.sum(np.diff(np.sign(train_msGyro), axis=1) != 0, axis=1)


In [8]:
# Testing Accelerometer features
test_features[:, 0, :] = np.mean(test_msAcc, axis=1)
test_features[:, 1, :] = np.max(test_msAcc, axis=1)
test_features[:, 2, :] = np.min(test_msAcc, axis=1)
test_features[:, 3, :] = np.std(test_msAcc, axis=1)
test_features[:, 4, :] = np.percentile(test_msAcc, 20, axis=1)
test_features[:, 5, :] = np.percentile(test_msAcc, 50, axis=1)
test_features[:, 6, :] = np.percentile(test_msAcc, 80, axis=1)
test_features[:, 7, :] = np.sum(np.diff(np.sign(test_msAcc), axis=1) != 0, axis=1)

# Testing Gyroscope features
test_features[:, 8, :] = np.mean(test_msGyro, axis=1)
test_features[:, 9, :] = np.max(test_msGyro, axis=1)
test_features[:, 10, :] = np.min(test_msGyro, axis=1)
test_features[:, 11, :] = np.std(test_msGyro, axis=1)
test_features[:, 12, :] = np.percentile(test_msGyro, 20, axis=1)
test_features[:, 13, :] = np.percentile(test_msGyro, 50, axis=1)
test_features[:, 14, :] = np.percentile(test_msGyro, 80, axis=1)
test_features[:, 15, :] = np.sum(np.diff(np.sign(test_msGyro), axis=1) != 0, axis=1)




In [9]:
print("Train Features Shape:", train_features.shape)
print("Test Features Shape:", test_features.shape)


Train Features Shape: (87, 16, 3)
Test Features Shape: (90, 16, 3)


In [10]:
# Reshape train features
train_features_reshaped = np.reshape(
    train_features, (train_features.shape[0], train_features.shape[1] * train_features.shape[2])
)
print("Reshaped Train Features Shape:", train_features_reshaped.shape)
# Reshape test features
test_features_reshaped = np.reshape(
    test_features, (test_features.shape[0], test_features.shape[1] * test_features.shape[2])
)
print("Reshaped Test Features Shape:", test_features_reshaped.shape)


Reshaped Train Features Shape: (87, 48)
Reshaped Test Features Shape: (90, 48)


In [124]:
##without using any batches the accuracy was varying from 65 to 81 which is very inconsistent##

# from os import times_result

# # Map 20 to 0 and 36 to 1
# train_labels_binary = np.where(train_labels == 20, 0, 1)
# test_labels_binary = np.where(test_labels == 20, 0, 1)

# import torch.optim as optim

# # Assuming train_features_reshaped and test_features_reshaped are your input data
# # Convert training and testing data to PyTorch tensors
# X_train_tensor = torch.tensor(train_features_reshaped, dtype=torch.float32)
# y_train_tensor = torch.tensor(train_labels_binary, dtype=torch.float32).unsqueeze(1)
# X_test_tensor = torch.tensor(test_features_reshaped, dtype=torch.float32)
# y_test_tensor = torch.tensor(test_labels_binary, dtype=torch.float32).unsqueeze(1)

# class BinaryClassificationModel(nn.Module):
#     def __init__(self, input_size):
#         super(BinaryClassificationModel, self).__init__()
#         self.hidden1 = nn.Linear(input_size, 128)  # First hidden layer with 128 neurons
#         self.hidden2 = nn.Linear(128, 64)         # Second hidden layer with 64 neurons
#         self.hidden3 = nn.Linear(64, 32)          # Third hidden layer with 32 neurons
#         self.output = nn.Linear(32, 1)            # Output layer
#         self.sigmoid = nn.Sigmoid()               # Sigmoid activation for binary classification

#     def forward(self, x):
#         x = torch.relu(self.hidden1(x))           # Apply ReLU activation to first hidden layer
#         x = torch.relu(self.hidden2(x))           # Apply ReLU activation to second hidden layer
#         x = torch.relu(self.hidden3(x))           # Apply ReLU activation to third hidden layer
#         x = self.sigmoid(self.output(x))          # Sigmoid activation for the output layer
#         return x

# # Initialize the improved model
# input_size = train_features_reshaped.shape[1]
# print(input_size)
# model = BinaryClassificationModel(input_size)

# # Define the loss function and optimizer
# criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
# optimizer = optim.SGD(model.parameters(), lr=0.01)  # Lowered learning rate to 0.1 for better convergence

# # Training the improved model
# num_epochs = 1000
# losses = []
# for epoch in range(num_epochs):
#     # Forward pass
#     outputs = model(X_train_tensor)
#     loss = criterion(outputs, y_train_tensor)

#     # Backward pass and optimization
#     optimizer.zero_grad()  # Clear old gradients
#     loss.backward()        # Compute new gradients
#     optimizer.step()       # Update model parameters

#     losses.append(loss.item())

#     # Print loss every 100 epochs
#     if (epoch + 1) % 100 == 0:
#         print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

# with torch.no_grad():
#     y_pred = model(X_test_tensor)
#     y_pred_classes = (y_pred > 0.5).float()  # Convert probabilities to binary labels
#     accuracy = (y_pred_classes == y_test_tensor).sum().item() / y_test_tensor.size(0)

# print(f"Test Accuracy: {accuracy * 100:.2f}%")

# from sklearn.metrics import classification_report, confusion_matrix
# print("Classification Report:")
# print(classification_report(y_test_tensor, y_pred_classes))
# print("Confusion Matrix:")
# print(confusion_matrix(y_test_tensor, y_pred_classes))


##by using batch##
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Mapping 20 to 0 and 36 to 1
train_labels_binary = np.where(train_labels == 20, 0, 1)
test_labels_binary = np.where(test_labels == 20, 0, 1)

# Converting training and testing data to PyTorch tensors
X_train_tensor = torch.tensor(train_features_reshaped, dtype=torch.float32)
y_train_tensor = torch.tensor(train_labels_binary, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(test_features_reshaped, dtype=torch.float32)
y_test_tensor = torch.tensor(test_labels_binary, dtype=torch.float32).unsqueeze(1)

# Creating DataLoader for batching
batch_size = 100  # Settig my batch size
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationModel, self).__init__()
        self.hidden1 = nn.Linear(input_size, 64)  # First hidden layer with 64 neurons
        self.hidden2 = nn.Linear(64, 32)         # Second hidden layer with 32 neurons
        self.output = nn.Linear(32, 1)           # Output layer
        self.sigmoid = nn.Sigmoid()              # Sigmoid activation for binary classification

    def forward(self, x):
        x = torch.relu(self.hidden1(x))          # Apply ReLU activation to the first hidden layer
        x = torch.relu(self.hidden2(x))          # Apply ReLU activation to the second hidden layer
        x = self.sigmoid(self.output(x))         # Sigmoid activation for the output layer
        return x



# Initializing the model
input_size = train_features_reshaped.shape[1]
model = BinaryClassificationModel(input_size)

# Defining the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training the model with batches
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()  # Setting the model to training mode
    total_loss = 0

    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Printing average loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Evaluating the model
model.eval()  # Setting the model to evaluation mode
y_pred_classes_list = []
y_test_list = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        preds = (outputs > 0.5).float()  # Converting probabilities to binary labels
        y_pred_classes_list.append(preds)
        y_test_list.append(batch_y)

# Concatenatinating all predictions and true labels
y_pred_classes = torch.cat(y_pred_classes_list)
y_test_tensor = torch.cat(y_test_list)

# Calculating accuracy
accuracy = (y_pred_classes == y_test_tensor).sum().item() / y_test_tensor.size(0)
print(f"Test Accuracy using loop: {accuracy * 100:.2f}%")

print("________________________________________________________")

from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(y_test_tensor, y_pred_classes))
print("Confusion Matrix:")
print(confusion_matrix(y_test_tensor, y_pred_classes))


Epoch 100/1000, Loss: 0.6004
Epoch 200/1000, Loss: 0.5461
Epoch 300/1000, Loss: 0.5228
Epoch 400/1000, Loss: 0.5024
Epoch 500/1000, Loss: 0.4838
Epoch 600/1000, Loss: 0.4616
Epoch 700/1000, Loss: 0.4495
Epoch 800/1000, Loss: 0.4338
Epoch 900/1000, Loss: 0.4271
Epoch 1000/1000, Loss: 0.4303
Test Accuracy using loop: 82.22%
________________________________________________________
Classification Report:
              precision    recall  f1-score   support

         0.0       0.77      0.96      0.85        48
         1.0       0.93      0.67      0.78        42

    accuracy                           0.82        90
   macro avg       0.85      0.81      0.81        90
weighted avg       0.84      0.82      0.82        90

Confusion Matrix:
[[46  2]
 [14 28]]


In [65]:
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_features_reshaped)
X_test_scaled = scaler.transform(test_features_reshaped)

# Initializing the Support Vector Classifier (SVM)
svm_model = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)

# Training the model
svm_model.fit(X_train_scaled, train_labels_binary)

# Making predictions on the test set
y_pred = svm_model.predict(X_test_scaled)

# Evaluating the model: Accuracy
accuracy = accuracy_score(test_labels_binary, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Printing classification report
print("Classification Report:")
print(classification_report(test_labels_binary, y_pred))

# Printing confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_labels_binary, y_pred))

Test Accuracy: 83.33%
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.90      0.85        48
           1       0.86      0.76      0.81        42

    accuracy                           0.83        90
   macro avg       0.84      0.83      0.83        90
weighted avg       0.84      0.83      0.83        90

Confusion Matrix:
[[43  5]
 [10 32]]
